In [25]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
############## sys imports #############
import os
import sys
import time
import copy
import argparse
import datetime
############## basic stats imports #############
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
############## pytorch imports #############
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
############## custom imports #############
from dataloader import FaceScrubDataset, TripletFaceScrub, SiameseFaceScrub
from dataloader import FaceScrubBalancedBatchSampler

from networks import *
from losses import OnlineTripletLoss
from openface.loadOpenFace import prepareOpenFace
from utils import save_checkpoint, save_hyperparams, AverageMeter, HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector

In [27]:
DATA_PATH = '/home/s1791387/facescrub-data/new_data_max/'
TRAIN_PATH = os.path.join(DATA_PATH, 'train_full_with_ids.txt')
VALID_PATH = os.path.join(DATA_PATH, 'val_full_with_ids.txt')
TEST_PATH = os.path.join(DATA_PATH, 'test_full_with_ids.txt')
WEIGHTS_PATH = '/home/s1791387/facescrub-data/new_data_max/openface_model_weigths/job_semi_std_cos3_Jul_25_1000hrs/weights_75.pth'

In [28]:
batch_size = 64
input_size = 96
output_dim = 128
learning_rate = 1e2
num_epochs = 1
start_epoch = 0

triplet_margin = 1.  # margin
triplet_p = 2  # norm degree for distance calculation

resume_training = True
workers = 4
use_cuda = True

In [29]:
cuda = False
pin_memory = False
if use_cuda and torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    cuda = True
    cudnn.benchmark = True
    pin_memory = True
else:
    device = torch.device("cpu")

print('Device set: {}'.format(device))
print('Training set path: {}'.format(TRAIN_PATH))
print('Training set Path exists: {}'.format(os.path.isfile(TRAIN_PATH)))

Device set: cuda
Training set path: /home/s1791387/facescrub-data/new_data_max/train_full_with_ids.txt
Training set Path exists: True


In [30]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
}


train_df = FaceScrubDataset(
    txt_file=TRAIN_PATH, root_dir=DATA_PATH, transform=data_transforms['val'])

val_df = FaceScrubDataset(
    txt_file=VALID_PATH, root_dir=DATA_PATH, transform=data_transforms['val'])

train_loader=torch.utils.data.DataLoader(
        train_df, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, num_workers=workers)
print('Train loader created. Length of train loader: {}'.format(
        len(train_loader)))
    
val_loader=torch.utils.data.DataLoader(
        val_df, batch_size=batch_size, shuffle=False, pin_memory=pin_memory, num_workers=workers)
print('Val loader created. Length of train loader: {}'.format(
        len(val_loader)))



Train loader created. Length of train loader: 860
Val loader created. Length of train loader: 92


In [31]:
openface = prepareOpenFace(useCuda=cuda)
params = sum(p.numel() for p in openface.parameters() if p.requires_grad)
print('Number of params in network {}'.format(params))
# inception=models.inception_v3(pretrained=True)
# inception.aux_logits=False
# num_ftrs=inception.fc.in_features
# inception.fc=nn.Linear(num_ftrs, output_dim)

# criterion=nn.TripletMarginLoss(margin=triplet_margin, p=triplet_p)

optimizer=optim.Adam(inception.parameters(), lr=learning_rate)

# scheduler=lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Sent model to GPU
Number of params in network 3733968


In [32]:
num_train_classes = len(train_df.pids)

In [33]:
classifier = ClassNet(input_size=output_dim, output_size=num_train_classes, training=True)

In [34]:
if resume_training:
    resume_weights=WEIGHTS_PATH
    if cuda:
        checkpoint=torch.load(resume_weights)
    else:
        # Load GPU model on CPU
        checkpoint=torch.load(resume_weights,
                                map_location=lambda storage,
                                loc: storage)

    start_epoch=checkpoint['epoch']
    openface.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_loss = checkpoint['best_loss']
    # scheduler.load_state_dict(checkpoint['scheduler'])
    print("=> loaded checkpoint '{}' (trained for {} epochs)".format(
        resume_weights, checkpoint['epoch']))
    for epoch in range(0, start_epoch):
        scheduler.step()

NameError: name 'optimizer' is not defined

In [ ]:
if cuda:
    openface.cuda()
    print('Sent model to gpu {}'.format(
        next(openface.parameters()).is_cuda))

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for batch_idx, (imgs, _) in enumerate(train_loader):
        data_time.update(time.time() - end)
        imgs = [img.to(device) for img in imgs]

        embed_anchor, embed_pos, embed_neg=model(imgs[0], imgs[1], imgs[2])
        loss = criterion(embed_anchor, embed_pos, embed_neg)

        losses.update(loss.item(), imgs[0].size(0))


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if batch_idx % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(
                   epoch, batch_idx, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
    return losses.avg

def validate(val_loader, model, criterion, device):
    batch_time = AverageMeter()
    losses = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (imgs, _) in enumerate(val_loader):
            imgs = [img.to(device) for img in imgs]

            embed_anchor, embed_pos, embed_neg=model(imgs[0], imgs[1], imgs[2])

            loss = criterion(embed_anchor, embed_pos, embed_neg)

            losses.update(loss.item(), imgs[0].size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(
                       i, len(val_loader), batch_time=batch_time, loss=losses))

    return losses.avg

In [ ]:
def tsne(embeddings):
    import sklearn.manifold
    return torch.from_numpy(sklearn.manifold.TSNE(n_iter = 250).fit_transform(embeddings.numpy()))

In [ ]:
def svg(points, labels, thumbnails, legend_size = 1e-1, legend_font_size = 5e-2, circle_radius = 5e-3):
	points = (points - points.min(0)[0]) / (points.max(0)[0] - points.min(0)[0])
	class_index = sorted(set(labels))
	class_colors = [360.0 * i / len(class_index) for i in range(len(class_index))]
	colors = [class_colors[class_index.index(label)] for label in labels]
	thumbnails_base64 = [base64.b64encode(cv2.imencode('.jpg', img.mul(255).permute(1, 2, 0).numpy()[..., ::-1])[1]) for img in thumbnails]
	return '<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 1 1">' + \
	   ''.join(map('''<circle cx="{}" cy="{}" title="{}" fill="hsl({}, 50%, 50%)" r="{}" desc="data:image/jpeg;base64,{}" onmouseover="evt.target.ownerDocument.getElementById('preview').setAttribute('href', evt.target.getAttribute('desc')); evt.target.ownerDocument.getElementById('label').textContent = evt.target.getAttribute('title');" />'''.format, points[:, 0], points[:, 1], labels, colors, [circle_radius] * len(points), thumbnails_base64)) + \
	   '''<image id="preview" x="0" y="{legend_size}" width="{legend_size}" height="{legend_size}" />
	   <text id="label" x="0" y="{legend_size}" font-size="{legend_font_size}" />
	   </svg>'''.format(legend_size = legend_size, legend_font_size = legend_font_size)

In [ ]:
tsne_embeddings = tsne(train_embeddings)

In [ ]:
import cv2
import base64

In [ ]:
open('train_tsne.svg', 'w').write(svg(tsne_embeddings, person_id, thumbnails))

In [ ]:
train_embeddings = train_embeddings.numpy()
np.correlate(train_embeddings, gender)

In [ ]:
import sklearn

In [ ]:
sklearn.metrics.mutual_info_score(train_embeddings, gender, contingency=None)